In [9]:
import spacy
from spacy.tokens import DocBin
import json
from sklearn.model_selection import train_test_split

In [2]:
data = json.load(open("annotations.json"))

len(data)

48

In [12]:
def spacify(data):
    nlp = spacy.blank("id")
    db = DocBin()

    for text, annot in data:
        doc = nlp.make_doc(text)
        annot = annot["entities"]

        ents, ent_indices = [], []

        for start, end, label in annot:
            skip_entity = False
            for i in range(start, end):
                if i in ent_indices:
                    skip_entity = True
                    break
            if skip_entity:
                continue

            ent_indices = ent_indices + list(range(start, end))

            try:
                span = doc.char_span(start, end, label=label, alignment_mode="strict")
            except:
                continue

            if span is None:
                err = str([start, end]) + "  " + str(text) + "\n"
                print(err)
            else:
                ents.append(span)
        
        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass

    return db

In [13]:
train_data, test_data = train_test_split(data, test_size=0.2)
len(train_data), len(test_data)

(38, 10)

In [14]:
db = spacify(train_data)
db.to_disk("train_data.spacy")

db = spacify(test_data)
db.to_disk("test_data.spacy")

```cmd
py -m spacy train config.cfg --output output --paths.train train_data.spacy --paths.dev test_data.spacy --gpu-id 0
```